In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

#1st authentification
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles 
#authentication
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=6241901948-jgbd6m744b6l85jk8gpt924oqj1ik232.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
!pip3 install pydrive

     ------------------------------------ 987.4/987.4 kB 754.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.8/8.8 MB 994.6 kB/s eta 0:00:00
     ------------------------------------ 151.7/151.7 kB 696.6 kB/s eta 0:00:00
     ------------------------------------ 114.6/114.6 kB 957.1 kB/s eta 0:00:00
     ------------------------------------ 525.5/525.5 kB 610.4 kB/s eta 0:00:00
     ------------------------------------ 211.7/211.7 kB 919.7 kB/s eta 0:00:00
Using legacy 'setup.py install' for pydrive, since package 'wheel' is not installed.
  Running setup.py install for pydrive: started
  Running setup.py install for pydrive: finished with status 'done'

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
from moviepy.video.fx.all import crop
import moviepy.video.fx.all as vfx

import moviepy.editor as mpy
import pytube
import os
import random
import xlrd
import pandas as pd

import gspread
import pandas as pd
import numpy as np

from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import requests
from io import BytesIO



class MakingVideo():
    
    def __init__(self,upload=False, drive=None):
        # define the scope
        self.scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        self.json_file_name = 'splila-204506-3f4a019fbea2.json'
        self.sheet_name = '54 Devotees'
        self.upload=upload
        self.drive=drive

    
    def read_spreadsheet(self):
        # add credentials to the account
        creds = ServiceAccountCredentials.from_json_keyfile_name(self.json_file_name, self.scope)

        # authorize the clientsheet 
        client = gspread.authorize(creds)

        # get the instance of the Spreadsheet
        sheet = client.open(self.sheet_name)

        # get the first sheet of the Spreadsheet
        sheet_instance = sheet.get_worksheet(0)

        # get all the records of the data
        records_data = sheet_instance.get_all_records()
        '''spreadsheetId = "1GkBTSYSSBYcTOhGA0FDv8hcAI0NryiDo"  # <--- Please set the Spreadsheet ID.

        # 1. Download the Google Spreadsheet as XLSX format.
        gauth = GoogleAuth()
        gauth.LocalWebserverAuth()
        url = "https://www.googleapis.com/drive/v3/files/" + spreadsheetId + "/export?mimeType=application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"
        res = requests.get(url, headers={"Authorization": "Bearer " + gauth.attr['credentials'].access_token})

        # 2. The downloaded XLSX data is read with `pd.read_excel`.
        sheet = "Videos"
        records_df = pd.read_excel(BytesIO(res.content), usecols=None, sheet_name=sheet)'''

        #convert the json to dataframe
        records_df = pd.DataFrame.from_dict(records_data)

        # run with yes records
        self.run_records = records_df[records_df['Run']=='Yes']
        pass
    def download_video(self, record):
        link = record['YouTube link']
        print("YouTube Link - "+link)
        self.folder_name = str(record['Posting Day'])+'_'+record['Name']
        working_dir = os.getcwd()
        self.directory = os.path.join(working_dir,self.folder_name)
        self.video_path=os.path.join(self.directory,self.folder_name)
        self.video_clip = self.video_path+'_clip.mp4'
        if not os.path.exists(self.directory):
            os.makedirs(self.directory)
        self.video_name = self.video_path+'.mp4'
        print(record['Name'])
        try:
            yt = pytube.YouTube(link)
        except:
            print('Youtube link is not working')
        try:
            if os.path.isfile(self.video_name):
                print('Downloaded video exists')
            else:
                yt.streams.filter(progressive=True, file_extension='mp4').get_highest_resolution().download(filename=self.video_name)
        except:
            print('Download was not successful')
        pass
    def get_clip(self,record):        
        #get_cuts
        cut_list=[record["Start"],record["End"],record["Start2"],record["End2"],record["Start3"],record["End3"],record["Start4"],record["End4"],record["Start5"],record["End5"] ]
        nonEmpty_cut_list = [i for i in cut_list if i] 
        cuts_topple=[]
        it=iter(nonEmpty_cut_list)
        for x in it:
            cuts_topple.append((x,next(it)))
        print(cuts_topple)    
        video= mpy.VideoFileClip(self.video_name)
        clips = []
        #clips.append(intro_subclip)
        for i,cut in enumerate(cuts_topple):
            start=float(str(cut[0]).split(".")[0])*60 + float(str(format(cut[0],'.2f')).split(".")[1])
            end=float(str(cut[1]).split(".")[0])*60 + float(str(format(cut[1],'.2f')).split(".")[1])
            if(len(str(cut[0]).split("."))==3):
                start = start*60+float(str(cut[0]).split(".")[2])
                end = end*60+float(str(cut[1]).split(".")[2])
            
            clip = video.subclip(start, end )
            #clip.set_fps(25.0)
            clips.append(clip)        
        final_clip = mpy.concatenate_videoclips(clips,method="compose")        
        
        final_clip.write_videofile(self.video_clip,fps=25)
        video.close()
        
    def crop_clip(self,crop_x=0,crop_y=0, crop_w=0,crop_h=0):
        if crop_x and crop_y and crop_w and crop_h:
            a,b,c,d= crop_x, crop_y, crop_w, crop_h
        else:
            video = cv2.VideoCapture(self.video_clip) 
            ret,frame= video.read()
            face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
            width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
            height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
            print("Original width "+str(width)+" and height "+str(height))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
            faces = face_cascade.detectMultiScale(gray, 1.1, 4) 
            max_i=0
            max_a=0
            for i,(x, y, w, h) in enumerate(faces):

                if  h*w > max_a:
                    max_i=i
                    max_a= h*w
                    x_, y_, w_, h_=x, y, w, h
            x, y, w, h = faces[max_i][0],faces[max_i][1],faces[max_i][2],faces[max_i][3] 
            max_l=max(w,h)
            cut_x = max(0,x-int(0.75*max_l))
            cut_y = max(0,y-int(0.75*max_l))
            cut_w = min(x+int(max_l+0.75*max_l),width)
            cut_h = min(y+int(max_l+0.75*max_l),height)

            #Replacing with input arguments
            cut_x = crop_x if crop_x else cut_x
            cut_y = crop_y if crop_y else cut_y
            cut_w = crop_w if crop_w else cut_w
            cut_h = crop_h if crop_h else cut_h
            a,b,c,d= cut_x, cut_y, cut_w, cut_h
            
        print("Cropping at point x "+str(a)+" point y "+str(b)+" width "+str(c)+" and height at "+str(d))

        self.video_cropped = self.video_path+'_crop.mp4'
        video2 = mpy.VideoFileClip(self.video_clip)
        video_final= crop(video2, x1=a, y1=b, x2=c, y2=d)

        video_final.write_videofile(
            self.video_cropped ,
            fps=30,
            remove_temp=True,
            codec="libx264",
            audio_codec="aac",
            threads = 6,
        )
        video2.close()


    def add_blur_background_logo_strip(self, pad=0, cropping_failed = False ,each_record=None):
        if cropping_failed:
            self.video_cropped = self.video_path+'_clip.mp4'
        else:
            self.video_cropped = self.video_path+'_crop.mp4'
        self.video_edited_final = self.video_path+'blur_background_logo_strip_added.mp4'
        with mpy.VideoFileClip(self.video_cropped) as video:
            if video.aspect_ratio < 1.8:
                video.write_videofile(
                    self.video_edited_final,
                    #ffmpeg_params=['-lavfi', '[0:v]scale=-1:ih*16/9,boxblur=luma_radius=min(h\,w)/20:luma_power=1:chroma_radius=min(cw\,ch)/20:chroma_power=1[bg];[bg][0:v]overlay=(W-w)/2:(H-h)/2,crop=h=iw*9/16']
                    #ffmpeg_params=["-lavfi", "[0:v] scale=350:640,boxblur=5[bg];[0:v]scale=350:500,pad=w=350:h=510:x=0:y=5:color=black [paded];[bg][paded]overlay=trunc((W-w)/2):trunc((H-h)/2)[prefinal];movie=logo_final2.png [watermark]; [prefinal][watermark] overlay=main_w-overlay_w-5:10"]
                    ffmpeg_params=["-lavfi", "[0:v] scale=350:640,boxblur=4[bg];[0:v]scale=350:520,pad=350:"+str(520+2*pad)+":0:"+str(pad)+":color=black [paded];[bg][paded]overlay=(W-w)/2:(H-h)/2[prefinal];movie=logo_final2.png [watermark]; [prefinal][watermark] overlay=main_w-overlay_w-5:6"]
                    
                    #ffmpeg_params=["-lavfi", "[0:v]scale=-1:720[scaled_video];[1:v]scale=1280:720,boxblur=50[blur_image];[blur_image][scaled_video]overlay=(main_w-overlay_w)/2:(main_h-overlay_h)/2[outv]"]
                )
        '''
        clips=[]
        clips.append(video)
        working_dir = os.getcwd()
        intro_fname = str(each_record['Intro'])+".mp4"
        Intro_video_name = os.path.join(working_dir,"Outro_short",intro_fname)
        intro_clip= mpy.VideoFileClip(Intro_video_name).resize( (350,640) )
        intro_subclip=intro_clip.set_fps(25.0)
        clips.append(intro_clip)
        
        Outro_video_name = os.path.join(working_dir,"Outro_short",intro_fname)
        Outro_clip= mpy.VideoFileClip(Outro_video_name).resize( (350,640) )
        Outro_clip=Outro_clip.set_fps(25.0)
        clips.append(Outro_clip)
        
        self.savetitle= self.video_path+'_into_outro_added.mp4'
        #final_clip = mpy.concatenate_videoclips(clips,method="compose")
        #final_clip= mpy.CompositeVideoClip([intro_subclip,video,Outro_clip])
        #final_clip.write_videofile(self.savetitle,fps=25)

        #video.close()
        '''
        
        main_video = mpy.VideoFileClip(self.video_edited_final)
        main_video=main_video.fx(vfx.resize, height=350, width=640).set_fps(25.0)
        
        working_dir = os.getcwd()
        intro_fname = str(each_record['Intro'])+".mp4"
        Intro_video_name = os.path.join(working_dir,"Intro",intro_fname)
        intro_clip= mpy.VideoFileClip(Intro_video_name)
        intro_clip =intro_clip.fx(vfx.resize, height=350, width=640).set_fps(25.0)
        
        Outro_video_name = os.path.join(working_dir,"Outro",intro_fname)
        Outro_clip= mpy.VideoFileClip(Outro_video_name)
        Outro_clip = Outro_clip.fx(vfx.resize, height=350, width=640).set_fps(25.0)
        
        self.savetitle= self.video_path+'_short.mp4'
        #final_video = mpy.CompositeVideoClip([intro_clip,main_video,Outro_clip])
        final_video = mpy.concatenate_videoclips([intro_clip,main_video,Outro_clip],method="compose")

        final_video.write_videofile(
            self.savetitle,
            fps=25,
            remove_temp=True,
            codec="libx264",
            audio_codec="aac",
            threads = 6,
        )
        
   
  
    def createDriveFolder(self, folderName, parentID = None):
        # Create a folder on Drive, returns the newely created folders ID
        body = {
          'title': folderName,
          'mimeType': "application/vnd.google-apps.folder"
        }
        if parentID:
            body['parents'] = [{'id': parentID}]
        root_folder = self.drive.files().insert(body = body).execute()
        return root_folder['id']
    
    def upload_files(self,each_record):
        [month, date]=each_record['Posting Day'].split()
        print("Saving in the month folder "+month+" and date folder "+date)
        fileList = self.drive.ListFile({'q': "'19SyToVl9Uik6DR0vC4RXRY66HO2C9lpp' in parents and trashed=false"}).GetList()
        for file in fileList:
            if(file['mimeType'] == "application/vnd.google-apps.folder") and file['title']==month:
                month_folder = self.drive.ListFile({'q': "'"+file['id']+"' in parents and trashed=false"}).GetList()
                print("Months folder ID "+file['id'])
                folder_ID=None
                for each_file in month_folder: 
                    if each_file['title']==date:
                        folder_ID = each_file['id']
                if not folder_ID:       
                    folder_ID = self.createDriveFolder(date,file['id'])
                print("Date folder ID "+folder_ID)
        
        file_metadata = {'title': 'Unedited_'+self.folder_name+'.mp4', "parents": [{"id": folder_ID, "kind": "drive#childList"}]}
        file1 = self.drive.CreateFile(metadata=file_metadata)
        file1.SetContentFile(self.savetitle)
        file1.Upload()
        
    def start(self):
        self.read_spreadsheet()
        for index,each_record in self.run_records.iterrows():
            self.download_video(each_record)
            self.get_clip(each_record)
            try:
                self.crop_clip()
                self.add_blur_background_logo_strip(5,False,each_record)
            except Exception as e:
                print(e)                
                self.add_blur_background_logo_strip(5,True,each_record)
            if self.upload:
                print("Uploading the files")
                self.upload_files(each_record)
        
        
if __name__ == "__main__":
    upload= True
    cl_O=MakingVideo(upload, drive)
    cl_O.start()   
        

StopIteration: 

In [6]:
cl_O.video_cropped = cl_O.video_path+'_crop.mp4'
video2 = mpy.VideoFileClip(cl_O.video_clip)
video_final= crop(video2, x1=170, y1=40, x2=400, y2=300)

video_final.write_videofile(
    cl_O.video_cropped ,
    fps=30,
    remove_temp=True,
    codec="libx264",
    audio_codec="aac",
    threads = 6,
)
video2.close()

Moviepy - Building video C:\Users\rnand\Desktop\nithya\videos SPA\August 10_Achyutananda Dasa\August 10_Achyutananda Dasa_crop.mp4.
MoviePy - Writing audio in August 10_Achyutananda Dasa_cropTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video C:\Users\rnand\Desktop\nithya\videos SPA\August 10_Achyutananda Dasa\August 10_Achyutananda Dasa_crop.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\rnand\Desktop\nithya\videos SPA\August 10_Achyutananda Dasa\August 10_Achyutananda Dasa_crop.mp4


In [5]:
for index,record in cl_O.run_records.iterrows():
        cut_list=[record["Start"],record["End"]]#,record["Start2"],record["End2"],record["Start3"],record["End3"],record["Start4"],record["End4"],record["Start5"],record["End5"] ]
        nonEmpty_cut_list = [i for i in cut_list if i] 
        cuts_topple=[]
        it=iter(nonEmpty_cut_list)
        for x in it:
            cuts_topple.append((x,next(it)))
        print(cuts_topple)    
        
        for i,cut in enumerate(cuts_topple):
            start=float(format(str(cut[0]).split(".")[0],'%.2f')*60 + float(str(cut[0]).split(".")[1])
            end=float(str(cut[1]).split(".")[0])*60 + float(str(cut[1]).split(".")[1])
            if(len(str(cut[0]).split("."))==3):
                start = start*60+float(str(cut[0]).split(".")[2])
                end = end*60+float(str(cut[1]).split(".")[2])

[(11.14, 11.2)]


In [10]:
float(str().split(".")[1])

20.0

In [8]:
str(cut[1]).split(".")[1]

'2'

In [ ]:
from mimetypes import MimeTypes
from googleapiclient.http import MediaFileUpload

mimetype = MimeTypes().guess_type(cl_O.savetitle)[0]

from googleapiclient.discovery import build
service = build('drive', 'v3', credentials=self.creds)


media = MediaFileUpload(filepath, mimetype=mimetype)
              
# Create a new file in the Drive storage
file = cl_O.drive.files().create(
    body=file_metadata, media_body=media, fields='id')

In [ ]:

folder_id="11QFUz_IhNIWtIk7vo4UyqKcOuqxaRd62"
file_metadata = {'title': 'sample.txt', "parents": [{"id": folder_id, "kind": "drive#childList"}]}
#file_metadata = {"parents": ["id":"19SyToVl9Uik6DR0vC4RXRY66HO2C9lpp","kind": "drive#file"]}
file1 = drive.CreateFile(metadata=file_metadata)
file1.SetContentFile('big.txt')
file1.Upload()

In [ ]:
fileList = drive.ListFile({'q': "'19SyToVl9Uik6DR0vC4RXRY66HO2C9lpp' in parents and trashed=false"}).GetList()
for file in fileList:
    if(file['mimeType'] == "application/vnd.google-apps.folder") and file['title']=="July":
        month_folder = drive.ListFile({'q': "'"+file['id']+"' in parents and trashed=false"}).GetList()
        for each_file in month_folder: 
            print('Title: %s, ID: %s AND mimeType: %s \n\n' % (each_file['title'], each_file['id'], each_file['mimeType']))

In [ ]:
"'"+file['id']+"' in parents and trashed=false"

In [23]:
cl_O.read_spreadsheet()
for index,each_record in cl_O.run_records.iterrows():
    cl_O.download_video(each_record)
    #cl_O.get_clip(each_record)
    record =each_record
    cut_list=[record["Start"],record["End"],record["Start2"],record["End2"],record["Start3"],record["End3"],record["Start4"],record["End4"],record["Start5"],record["End5"] ]
    nonEmpty_cut_list = [i for i in cut_list if i] 
    cuts_topple=[]
    it=iter(nonEmpty_cut_list)
    for x in it:
        cuts_topple.append((x,next(it)))
    print(cuts_topple)    
    video= mpy.VideoFileClip(cl_O.video_name)
    clips = []
    #clips.append(intro_subclip)
    for i,cut in enumerate(cuts_topple):
        start=float(cut[0].split(":")[1])*60 + float(cut[0].split(":")[2])
        end=float(cut[1].split(":")[1])*60 + float(cut[1].split(":")[2])
        clip = video.subclip(start, end )
        #clip.set_fps(25.0)
        clips.append(clip)        
    final_clip = mpy.concatenate_videoclips(clips,method="compose")        
    cl_O.video_clip = cl_O.video_path+'_edited.mp4'
    final_clip.write_videofile(cl_O.video_clip,fps=25)
    video.close()

YouTube Link - https://www.youtube.com/watch?v=HaOLZ0VmoCs
Tranakarta Dasa
[(35.3, 36.1)]


AttributeError: 'float' object has no attribute 'split'

In [ ]:
start=float(cut[0].split(":")[1])*60 + float(cut[0].split(":")[2])

In [ ]:
float(cut[0].split(":")[1])*60

In [ ]:
cut[0].split(":")

In [ ]:
crop_w=50
crop_h=0
max_l =50
x=20
y=20
width = 100
height = 150
cut_w = min(x+int(max_l+max_l),width)
cut_h = min(y+int(max_l+max_l),height)
cut_w = crop_w if crop_w else cut_w
cut_h = crop_h if crop_h else cut_h
print(cut_w,cut_h)

In [ ]:
video.get(cv2.CAP_PROP_FRAME_WIDTH)
video.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:

import moviepy.editor as mpy

cl_O.video_cropped = cl_O.video_path+'_cropped.mp4'
video_edited_final = cl_O.video_path+'blur.mp4'
pad=6
with mpy.VideoFileClip(cl_O.video_cropped) as video:
    if video.aspect_ratio < 1.8:
        video.write_videofile(
            video_edited_final,
            #ffmpeg_params=['-lavfi', '[0:v]scale=-1:ih*16/9,boxblur=luma_radius=min(h\,w)/20:luma_power=1:chroma_radius=min(cw\,ch)/20:chroma_power=1[bg];[bg][0:v]overlay=(W-w)/2:(H-h)/2,crop=h=iw*9/16']
            #ffmpeg_params=["-lavfi", "[0:v] scale=350:640,boxblur=5[bg];[0:v]scale=350:500,pad=w=350:h=510:x=0:y=5:color=black [paded];[bg][paded]overlay=trunc((W-w)/2):trunc((H-h)/2)[prefinal];movie=logo_final2.png [watermark]; [prefinal][watermark] overlay=main_w-overlay_w-5:10"]
            ffmpeg_params=["-lavfi", "[0:v] scale=350:640,boxblur=4[bg];[0:v]scale=350:520,pad=350:"+str(520+2*pad)+":0:"+str(pad)+":color=black [paded];[bg][paded]overlay=(W-w)/2:(H-h)/2[prefinal];movie=logo_final2.png [watermark]; [prefinal][watermark] overlay=main_w-overlay_w-5:6"]

            #ffmpeg_params=["-lavfi", "[0:v]scale=-1:720[scaled_video];[1:v]scale=1280:720,boxblur=50[blur_image];[blur_image][scaled_video]overlay=(main_w-overlay_w)/2:(main_h-overlay_h)/2[outv]"]
        )

In [ ]:
"-lavfi", "[0:v] scale=350:640,boxblur=4[bg];[0:v][0:v]scale=350:520,pad=350:532:0:"+str(6)+":color=black [paded];[bg][paded]overlay=(W-w)/2:(H-h)/2[prefinal];movie=logo_final2.png [watermark]; [prefinal][watermark] overlay=main_w-overlay_w-5:6".format(pad)